# Preprocessing for BBBC022
## Annotations
### CellProfiler Annotations: Three Class Experiment
- Use the CellProfiler annotations to create 3-class labels
- Boundary width of 2 is used

### Hand Annotations: Three Class Experiment
- Create 3-class labels from annotations for comparison with previous experiments
- Do this for different boundary widths (2, 4, 6 and 8)

### Hand Annotations: Boundary Experiment
- Create binary (hard) boundary images from annotations (different boundary sizes)
- Create soft boundary images from annotations (different smoothing filters)

### Sanity Check
Check results of previous preprocessing steps. Did we do the right thing?
## Image Preprocessing
- Give a preview of how the data should be processed in the generator preparing it for training (normalizing and converting to 8 bit)

## Splitting Up
- Split up into training, validation and test set

## Windowing
- Window each image to make it compatible with the existing data generators. In the future this is going to be done on the fly by a custom data generator.

### Sanity Check
Check results of previous preprocessing steps. Did we do the right thing?

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

import skimage.io
import skimage.segmentation
import skimage.filters

import scipy.stats

import random

import os
import shutil

import joblib
import warnings

In [ ]:
debug = False

# Parameters

In [ ]:
# assume a nucleus is at least 10 by 10 pixels big
min_nucleus_size = 10**2

In [ ]:
# general
dir_root = '/home/jr0th/github/segmentation/data/BBBC022/'

# split folders
dir_training = dir_root + 'training/'
dir_validation = dir_root + 'validation/'
dir_test = dir_root + 'test/'

# raw data
dir_raw_images = dir_root + 'raw/raw_images/'
dir_raw_annotations = dir_root + 'raw/raw_annotations/'
dir_raw_cp_segmentations = dir_root + 'raw/raw_cp_segmentations/'

path_files_training = dir_root + 'training.txt'
path_files_validation = dir_root + 'validation.txt'
path_files_test = dir_root + 'test.txt'

In [ ]:
## three class experiment - labels

# CP (binary image with boundary width of 2)
dir_label_cp = dir_root + 'y/label_cp/'

# hand annotations
dir_label_binary_2 = dir_root + 'y/label_binary_2/'
dir_label_binary_4 = dir_root + 'y/label_binary_4/'
dir_label_binary_6 = dir_root + 'y/label_binary_6/'
dir_label_binary_8 = dir_root + 'y/label_binary_8/'
dir_label_single_channel = dir_root + 'y/label_single_channel/'

## boudnary experiment
dir_boundary_hard_2 = dir_root + 'y/boundary_2/'
dir_boundary_hard_4 = dir_root + 'y/boundary_4/'
dir_boundary_hard_6 = dir_root + 'y/boundary_6/'
dir_boundary_hard_8 = dir_root + 'y/boundary_8/'
dir_boundary_soft = dir_root + 'y/boundary_soft/'

## input data, normalized and 8 bit
dir_images_normalized_8bit = dir_root + 'x/images_normalized_8bit/'

## CellProfiler Annotations: Three Class Experiment

In [ ]:
filelist = sorted(os.listdir(dir_raw_annotations))

# run over all raw images
for filename in filelist:
    
    # GET ANNOTATION
    cp_segm = skimage.io.imread(dir_raw_cp_segmentations + filename[:-3] + 'tiff')
    
    # filter small objects, e.g. micronulcei
    cp_segm = skimage.morphology.remove_small_objects(cp_segm, min_size=min_nucleus_size)
    
    # find boundaries
    boundaries = skimage.segmentation.find_boundaries(cp_segm)
    
    # prepare buffer for binary label
    label_cp = np.zeros((cp_segm.shape + (3,)))
    
    # write binary label
    label_cp[(cp_segm == 0) & (boundaries == 0), 0] = 1
    label_cp[(cp_segm != 0) & (boundaries == 0), 1] = 1
    label_cp[boundaries == 1, 2] = 1
    
    # save it - converts image to range from 0 to 255
    skimage.io.imsave(dir_label_cp + filename, label_cp)
    
    if(debug):
        print(cp_segm.dtype, cp_segm.shape)
        
        # plot original annotation
        plt.imshow(cp_segm)
        plt.colorbar()
        plt.show()
        
        print(label_cp.dtype)
        
        # plot binary label
        plt.figure()
        plt.imshow(label_cp)
        plt.show()

## Hand Annotations: Three Class Experiment

In [ ]:
filelist = sorted(os.listdir(dir_raw_annotations))

# run over all raw images
for filename in filelist:
    
    # GET ANNOTATION
    annot = skimage.io.imread(dir_raw_annotations + filename)
    
    # strip the first channel
    annot = annot[:,:,0]
    
    # label the annotations nicely to prepare for future filtering operation
    annot = skimage.morphology.label(annot)
    
    # filter small objects, e.g. micronulcei
    annot = skimage.morphology.remove_small_objects(annot, min_size=min_nucleus_size)
    
    # find boundaries
    boundaries_2 = skimage.segmentation.find_boundaries(annot)
    
    
    # BINARY LABEL - 2
    
    # prepare buffer for binary label
    label_binary_2 = np.zeros((annot.shape + (3,)))
    
    # write binary label
    label_binary_2[(annot == 0) & (boundaries_2 == 0), 0] = 1
    label_binary_2[(annot != 0) & (boundaries_2 == 0), 1] = 1
    label_binary_2[boundaries_2 == 1, 2] = 1
    
    # save it - converts image to range from 0 to 255
    skimage.io.imsave(dir_label_binary_2 + filename, label_binary_2)

    
    # BINARY LABEL - 4
    
    boundaries_4 = skimage.morphology.binary_dilation(boundaries_2)
    
    # prepare buffer for binary label
    label_binary_4 = np.zeros((annot.shape + (3,)))
    
    # write binary label
    label_binary_4[(annot == 0) & (boundaries_4 == 0), 0] = 1
    label_binary_4[(annot != 0) & (boundaries_4 == 0), 1] = 1
    label_binary_4[boundaries_4 == 1, 2] = 1
    
    # save it - converts image to range from 0 to 255
    skimage.io.imsave(dir_label_binary_4 + filename, label_binary_4)
    
    
    # BINARY LABEL - 6
    
    boundaries_6 = skimage.morphology.binary_dilation(boundaries_4)
    
    # prepare buffer for binary label
    label_binary_6 = np.zeros((annot.shape + (3,)))
    
    # write binary label
    label_binary_6[(annot == 0) & (boundaries_6 == 0), 0] = 1
    label_binary_6[(annot != 0) & (boundaries_6 == 0), 1] = 1
    label_binary_6[boundaries_6 == 1, 2] = 1
    
    # save it - converts image to range from 0 to 255
    skimage.io.imsave(dir_label_binary_6 + filename, label_binary_6)
    
    
    # BINARY LABEL - 8
    
    boundaries_8 = skimage.morphology.binary_dilation(boundaries_6)
    
    # prepare buffer for binary label
    label_binary_8 = np.zeros((annot.shape + (3,)))
    
    # write binary label
    label_binary_8[(annot == 0) & (boundaries_8 == 0), 0] = 1
    label_binary_8[(annot != 0) & (boundaries_8 == 0), 1] = 1
    label_binary_8[boundaries_8 == 1, 2] = 1
    
    # save it - converts image to range from 0 to 255
    skimage.io.imsave(dir_label_binary_8 + filename, label_binary_8)
    
    
    # SINGLE CHANNEL LABEL
    
    # prepare buffer for single channel label
    label_single_channel = np.zeros(annot.shape, dtype = np.uint8)
    
    # write binary label
    label_single_channel[(annot == 0) & (boundaries_2 == 0)] = 0
    label_single_channel[(annot != 0) & (boundaries_2 == 0)] = 1
    label_single_channel[boundaries_2 == 1] = 2
    
    # save it
    skimage.io.imsave(dir_label_single_channel + filename, label_single_channel)
    
    if(debug):
        print(annot.dtype, annot.shape)
        
        # plot original annotation
        plt.imshow(annot)
        plt.colorbar()
        plt.show()
        
        print(label_binary.dtype)
        
        # plot binary label
        plt.figure()
        plt.imshow(label_binary)
        plt.show()
        
        # plot single channel label
        plt.figure()
        plt.imshow(label_single_channel)
        plt.colorbar()
        plt.show()

## Hand Annotations: Boundary Experiment

In [ ]:
filelist = sorted(os.listdir(dir_raw_annotations))

# run over all raw images
for filename in filelist:
    
    # ANNOTATION
    annot = skimage.io.imread(dir_raw_annotations + filename)
    
    # strip the first channel
    annot = annot[:,:,0]
    
    # label the annotations nicely to prepare for future filtering operation
    annot = skimage.morphology.label(annot)
    
    # filter small objects, e.g. micronulcei
    annot = skimage.morphology.remove_small_objects(annot, min_size=min_nucleus_size)
    
    # find boundaries
    boundaries = skimage.segmentation.find_boundaries(annot)

    # BOUNDARY HARD 2
    boundary_hard_2 = skimage.segmentation.find_boundaries(annot)

    # save it - converts image to range from 0 to 255
    skimage.io.imsave(dir_boundary_hard_2 + filename, boundary_hard_2)
    
    # BOUNDARY HARD 4
    boundary_hard_4 = skimage.morphology.binary_dilation(boundary_hard_2)
    
    # save it - converts image to range from 0 to 255
    skimage.io.imsave(dir_boundary_hard_4 + filename, boundary_hard_4)

    # BOUNDARY HARD 6
    boundary_hard_6 = skimage.morphology.binary_dilation(boundary_hard_4)
    
    # save it - converts image to range from 0 to 255
    skimage.io.imsave(dir_boundary_hard_6 + filename, boundary_hard_6)

    # BOUNDARY HARD 8
    boundary_hard_8 = skimage.morphology.binary_dilation(boundary_hard_6)
    
    # save it - converts image to range from 0 to 255
    skimage.io.imsave(dir_boundary_hard_8 + filename, boundary_hard_8)
    
    # assume "real" boundary is at 0, imitate Gaussian distribution around clear boundary
    std = 1.5
    factor_8 = scipy.stats.norm.pdf(3.5, scale=std)
    factor_6 = scipy.stats.norm.pdf(2.5, scale=std) - factor_8
    factor_4 = scipy.stats.norm.pdf(1.5, scale=std) - factor_6 - factor_8
    factor_2 = scipy.stats.norm.pdf(0.5, scale=std) - factor_4 - factor_6 - factor_8
    
    boundary_soft_unscaled = \
        factor_2 * boundary_hard_2 + \
        factor_4 * boundary_hard_4 + \
        factor_6 * boundary_hard_6 + \
        factor_8 * boundary_hard_8
        
    # rescale to make ensure original boundary pixels get certainty of 1
    boundary_soft = boundary_soft_unscaled / scipy.stats.norm.pdf(0.5, scale=std)
    
    # make an image again - scale down to ubyte
    boundary_soft = skimage.img_as_ubyte(boundary_soft)
    
    # save it - converts image to range from 0 to 255
    skimage.io.imsave(dir_boundary_soft + filename, boundary_soft)
    
    if(debug):
        print(annot.dtype, annot.shape)
        
        # plot original annotation
        plt.imshow(annot)
        plt.colorbar()
        plt.show()
        
        # plot boundary hard 2
        plt.figure()
        plt.imshow(boundary_hard_2)
        plt.colorbar()
        plt.show()
        
        plt.figure()
        plt.imshow(boundary_hard_4)
        plt.colorbar()
        plt.show()
        
        plt.figure()
        plt.imshow(boundary_hard_6)
        plt.colorbar()
        plt.show()
        
        plt.figure()
        plt.imshow(boundary_hard_8)
        plt.colorbar()
        plt.show()
        
        plt.figure(figsize=(20,20))
        plt.imshow(boundary_soft)
        plt.colorbar()
        plt.show()
        

## Image Preprocessing

In [ ]:
filelist = sorted(os.listdir(dir_raw_images))

# run over all raw images
for filename in filelist:
    
    # load image and its annotation
    img = skimage.io.imread(dir_raw_images + filename)

    if(debug):
        print("BEFORE")
        print(img.dtype, img.shape)
        plt.imshow(img)
        plt.show()
        plt.hist(img.flatten())
        plt.show()        

    # IMAGE

    # normalize to [0,1]
    percentile = 99.9
    high = np.percentile(img, percentile)
    low = np.percentile(img, 100-percentile)

    img = np.minimum(high, img)
    img = np.maximum(low, img)

    img = (img - low) / (high - low) # gives float64, thus cast to 8 bit later
    img = skimage.img_as_ubyte(img) 
    
    skimage.io.imsave(dir_images_normalized_8bit + filename[:-3] + 'png', img)

    if(debug):
        print("AFTER")
        print(img.dtype, img.shape)
        plt.imshow(img)
        plt.show()
        plt.hist(img.flatten())
        plt.show()

## Splitting Up
- Split up the 200 images in training, validation and test sets
- Use the bash script 'setup_split_structure.sh' to set up the folder structure before

In [ ]:
with open(path_files_training) as f:
    training_files = f.read().splitlines()
with open(path_files_validation) as f:
    validation_files = f.read().splitlines()
with open(path_files_test) as f:
    test_files = f.read().splitlines()

In [ ]:
for filename in training_files:
    shutil.copyfile(dir_images_normalized_8bit + filename, dir_training + 'x_big/' + filename)
    shutil.copyfile(dir_label_cp + filename, dir_training + 'y_big_label_cp/' + filename)
    shutil.copyfile(dir_label_binary_2 + filename, dir_training + 'y_big_label_binary_2/' + filename)
    shutil.copyfile(dir_label_binary_4 + filename, dir_training + 'y_big_label_binary_4/' + filename)
    shutil.copyfile(dir_label_binary_6 + filename, dir_training + 'y_big_label_binary_6/' + filename)
    shutil.copyfile(dir_label_binary_8 + filename, dir_training + 'y_big_label_binary_8/' + filename)
    shutil.copyfile(dir_label_single_channel + filename, dir_training + 'y_big_label_single_channel/' + filename)
    shutil.copyfile(dir_boundary_hard_2 + filename, dir_training + 'y_big_boundary_2/' + filename)
    shutil.copyfile(dir_boundary_hard_4 + filename, dir_training + 'y_big_boundary_4/' + filename)
    shutil.copyfile(dir_boundary_hard_6 + filename, dir_training + 'y_big_boundary_6/' + filename)
    shutil.copyfile(dir_boundary_hard_8 + filename, dir_training + 'y_big_boundary_8/' + filename)
    shutil.copyfile(dir_boundary_soft + filename, dir_training + 'y_big_boundary_soft/' + filename)
    
for filename in validation_files:
    shutil.copyfile(dir_images_normalized_8bit + filename, dir_validation + 'x_big/' + filename)
    shutil.copyfile(dir_label_cp + filename, dir_validation + 'y_big_label_cp/' + filename)
    shutil.copyfile(dir_label_binary_2 + filename, dir_validation + 'y_big_label_binary_2/' + filename)
    shutil.copyfile(dir_label_binary_4 + filename, dir_validation + 'y_big_label_binary_4/' + filename)
    shutil.copyfile(dir_label_binary_6 + filename, dir_validation + 'y_big_label_binary_6/' + filename)
    shutil.copyfile(dir_label_binary_8 + filename, dir_validation + 'y_big_label_binary_8/' + filename)
    shutil.copyfile(dir_label_single_channel + filename, dir_validation + 'y_big_label_single_channel/' + filename)
    shutil.copyfile(dir_boundary_hard_2 + filename, dir_validation + 'y_big_boundary_2/' + filename)
    shutil.copyfile(dir_boundary_hard_4 + filename, dir_validation + 'y_big_boundary_4/' + filename)
    shutil.copyfile(dir_boundary_hard_6 + filename, dir_validation + 'y_big_boundary_6/' + filename)
    shutil.copyfile(dir_boundary_hard_8 + filename, dir_validation + 'y_big_boundary_8/' + filename)
    shutil.copyfile(dir_boundary_soft + filename, dir_validation + 'y_big_boundary_soft/' + filename)
    
for filename in test_files:
    shutil.copyfile(dir_images_normalized_8bit + filename, dir_test + 'x_big/' + filename)
    shutil.copyfile(dir_label_cp + filename, dir_test + 'y_big_label_cp/' + filename)
    shutil.copyfile(dir_label_binary_2 + filename, dir_test + 'y_big_label_binary_2/' + filename)
    shutil.copyfile(dir_label_binary_4 + filename, dir_test + 'y_big_label_binary_4/' + filename)
    shutil.copyfile(dir_label_binary_6 + filename, dir_test + 'y_big_label_binary_6/' + filename)
    shutil.copyfile(dir_label_binary_8 + filename, dir_test + 'y_big_label_binary_8/' + filename)
    shutil.copyfile(dir_label_single_channel + filename, dir_test + 'y_big_label_single_channel/' + filename)
    shutil.copyfile(dir_boundary_hard_2 + filename, dir_test + 'y_big_boundary_2/' + filename)
    shutil.copyfile(dir_boundary_hard_4 + filename, dir_test + 'y_big_boundary_4/' + filename)
    shutil.copyfile(dir_boundary_hard_6 + filename, dir_test + 'y_big_boundary_6/' + filename)
    shutil.copyfile(dir_boundary_hard_8 + filename, dir_test + 'y_big_boundary_8/' + filename)
    shutil.copyfile(dir_boundary_soft + filename, dir_test + 'y_big_boundary_soft/' + filename)

## Windowing
- Run through all created folders and window the images inside to decent sized patches

In [ ]:
def process_gray(index, images, out_dir_image):
    
    image = images[index]

    path = images.files[index]
    filename = os.path.basename(path)
    
    split = os.path.splitext(filename)

    blocks = skimage.util.view_as_windows(image, (256, 256), 256)
    
    for i in range(blocks.shape[0]):
        for j in range(blocks.shape[1]):
            patchname = split[0] + '_' + str(i) + '_' + str(j) + split[1]
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                skimage.io.imsave(out_dir_image + patchname, blocks[i,j])

In [ ]:
def process_rgb(index, labels, out_dir_label):
    
    label = labels[index]

    path = labels.files[index]
    filename = os.path.basename(path)
    
    split = os.path.splitext(filename)

    blocks = skimage.util.view_as_windows(label, (256, 256, 3), 256)
    blocks = blocks.squeeze()
    
    for i in range(blocks.shape[0]):
        for j in range(blocks.shape[1]):
            patchname = split[0] + '_' + str(i) + '_' + str(j) + split[1]
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                skimage.io.imsave(out_dir_label + patchname, blocks[i,j])

In [ ]:
def process_gray_dir(in_dir_gray, out_dir_gray):
    images = skimage.io.imread_collection(in_dir_gray + '*.png')
    joblib.Parallel(n_jobs = 100)(joblib.delayed(process_gray)(i, images, out_dir_gray) for i in range(len(images)))  
    
def process_rgb_dir(in_dir_rgb, out_dir_rgb):
    images = skimage.io.imread_collection(in_dir_rgb + '*.png')
    joblib.Parallel(n_jobs = 100)(joblib.delayed(process_rgb)(i, images, out_dir_rgb) for i in range(len(images)))  

In [ ]:
gray_folders_in = [
    'x_big/', 
    'y_big_label_single_channel/', 
    'y_big_boundary_2/', 
    'y_big_boundary_4/', 
    'y_big_boundary_6/', 
    'y_big_boundary_8/', 
    'y_big_boundary_soft/'
]

rgb_folders_in = [
    'y_big_label_cp/',
    'y_big_label_binary_2/',
    'y_big_label_binary_4/',
    'y_big_label_binary_6/',
    'y_big_label_binary_8/',
]

in_dirs_gray = [dir_training + folder for folder in gray_folders_in] + \
    [dir_validation + folder for folder in gray_folders_in] + \
    [dir_test + folder for folder in gray_folders_in]
in_dirs_rgb = [dir_training + folder for folder in rgb_folders_in] + \
    [dir_validation + folder for folder in rgb_folders_in] + \
    [dir_test + folder for folder in rgb_folders_in]
    
gray_folders_out = [
    'x/', 
    'y_label_single_channel/', 
    'y_boundary_2/', 
    'y_boundary_4/', 
    'y_boundary_6/', 
    'y_boundary_8/', 
    'y_boundary_soft/'
]

rgb_folders_out = [
    'y_label_cp/',
    'y_label_binary_2/',
    'y_label_binary_4/',
    'y_label_binary_6/',
    'y_label_binary_8/',
]

out_dirs_gray = [dir_training + folder + 'all/' for folder in gray_folders_out] + \
    [dir_validation + folder + 'all/' for folder in gray_folders_out] + \
    [dir_test + folder + 'all/' for folder in gray_folders_out]
out_dirs_rgb = [dir_training + folder + 'all/' for folder in rgb_folders_out] + \
    [dir_validation + folder + 'all/' for folder in rgb_folders_out] + \
    [dir_test + folder + 'all/' for folder in rgb_folders_out]

In [ ]:
for i in range(len(in_dirs_gray)):
    process_gray_dir(in_dirs_gray[i], out_dirs_gray[i])

In [ ]:
for i in range(len(in_dirs_rgb)):
    process_rgb_dir(in_dirs_rgb[i], out_dirs_rgb[i])

## Sanity Check
(pick some random examples)

In [ ]:
testfiles = [
    '/home/jr0th/github/segmentation/data/BBBC022/validation/x/all/IXMtest_A02_s1_w1051DAA7C-7042-435F-99F0-1E847D9B42CB_0_0.png',
    '/home/jr0th/github/segmentation/data/BBBC022/validation/y_boundary_2/all/IXMtest_B04_s4_w1F6AEFA0F-AF87-4B3B-A334-698647CFE043_0_0.png',
    '/home/jr0th/github/segmentation/data/BBBC022/validation/y_boundary_soft/all/IXMtest_B12_s2_w19F7E0279-D087-4B5E-9899-61971C29CB78_0_0.png',
    '/home/jr0th/github/segmentation/data/BBBC022/validation/y_label_single_channel/all/IXMtest_B17_s7_w1215A0A98-4A76-4846-B54A-F7C1EAF84E02_0_0.png',
    '/home/jr0th/github/segmentation/data/BBBC022/validation/y_label_binary_2/all/IXMtest_B19_s7_w1E43B84DB-39E2-4BFB-8CB4-554B32098C75_0_0.png',
    '/home/jr0th/github/segmentation/data/BBBC022/validation/y_label_cp/all/IXMtest_B24_s9_w18C4FE0DD-12CA-4711-9722-3E3105D1E691_0_0.png'
]

for testfile in testfiles:
    image = skimage.io.imread(testfile)

    print(testfile)
    print(image.dtype, image.shape)

    plt.hist(image.flatten())
    plt.show()

    plt.imshow(image)
    plt.colorbar()
    plt.show()